In [8]:
import os 
import json
import pprint as pp
import tweepy
import glob
import networkx as nx
import pandas as pd
from pyvis.network import Network

In [1]:
#Twitter API credentials
api_key = "M5NYCCuMTxLKm4S6DrsK5NqpR"
api_secret = "y3svwwgghPa9l0ieZReuFzuOPlkX92BrW4qfn4kwXAFoWffHzR"
access_token = "951194016212807680-RX1wBOiNZxau9k7nHyOvJk1mXNtyOp6"
access_secret = "Dt5x1PnX7jhsZvUCQq8gqlN0HDMjqVyxN8yb0To57uQEq"

In [16]:
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if(api.verify_credentials):
    print ('Authentication completed successfully!')

Authentication completed successfully!


# JSON

In [5]:
data_folder = "data"

def serialize_json(folder, filename, data):
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}


# FOLLOWERS e FOLLOWING

In [6]:
def findUsers(userlist,mode): #mode: 0 = followers, 1 = following
    quantity=20
    l2list = {}

    for user in userlist:
        print(f"Processing user {user}")

        found = []
        l2list[user]=[]

        if mode==0:
            usergroup=tweepy.Cursor(api.followers,screen_name = user ).items(quantity)
        else:
            usergroup=tweepy.Cursor(api.friends,screen_name = user ).items(quantity)

        for item in usergroup:

            json_data = item._json

            found_user = {}
            found_user["id"] = json_data["id"]
            found_user["name"] = json_data["name"]
            found_user["screen_name"] = json_data["screen_name"]
            found_user["location"] = json_data["location"]
            found_user["followers_count"] = json_data["followers_count"]
            found_user["friends_count"] = json_data["friends_count"]
            found_user["protected"] = json_data["protected"]


            if item.friends_count < 1000 and item.followers_count<1000 and json_data["protected"] == False and (found_user not in found):
                found.append(found_user)
                l2list[user].append(item.screen_name)
                print(item.screen_name)

            if len(found) >= 5:
                break


        if mode==0:
            serialize_json(data_folder+"/users", user+"_followers.json", found)
        else:
            serialize_json(data_folder+"/users", user+"_following.json", found)

    return l2list

#l1_followers=findUsers(["mizzaro","Miccighel_"],0)
l1_following=findUsers(["mizzaro","Miccighel_"],1)

#l2a_followers_mizzaro=findUsers(l1_followers["mizzaro"],0)
#l2a_following_mizzaro=findUsers(l1_followers["mizzaro"],1)

#l2a_followers_miccighel=findUsers(l1_followers["Miccighel_"],0)
#l2a_following_miccighel=findUsers(l1_followers["Miccighel_"],1)

l2b_followers_mizzaro=findUsers(l1_following["mizzaro"],0)
l2b_following_mizzaro=findUsers(l1_following["mizzaro"],1)

l2b_followers_miccighel=findUsers(l1_following["Miccighel_"],0)
l2b_following_miccighel=findUsers(l1_following["Miccighel_"],1)

Processing user mizzaro
MaxZancanaro
CIRES_AU
mccart_richard
Miccighel_
rmit_cidda
Data serialized to path: data/mizzaro_following.json
Processing user Miccighel_
fra_bomba6
Outsider289
BiancolinoElisa
ZuzanPink
eglu81
Data serialized to path: data/Miccighel__following.json
Processing user MaxZancanaro
gianmariomaggio
mizzaro
DanielaPetrell1
3DOMFBK
diunito
Data serialized to path: data/MaxZancanaro_followers.json
Processing user CIRES_AU
crc_gk
mizzaro
CGowlett1
linhlt262626
haskhosravi
Data serialized to path: data/CIRES_AU_followers.json
Processing user mccart_richard
johnedwardLEIG1
shelley_marshal
Nolongerscot
mizzaro
DrJJRich
Data serialized to path: data/mccart_richard_followers.json
Processing user Miccighel_
Il_Bale_
Outsider289
mizzaro
EUROARTISTS
fra_bomba6
Data serialized to path: data/Miccighel__followers.json
Processing user rmit_cidda
sumitsidana17
nowshed_bd
Dr_Hai_Dong
MaksudaBilkis
bitcoinub
Data serialized to path: data/rmit_cidda_followers.json
Processing user MaxZa

Rate limit reached. Sleeping for: 24


In [7]:
fileList=glob.glob(data_folder+"/users/*.json")

totalData=[]

for fileName in fileList:
    with open(fileName,'r',encoding="UTF-8") as file:
        data=json.load(file)
        #totalData.append(data)
        for user in data:
            totalData.append(user)

serialize_json(data_folder+"/total", "total_users.json", totalData)

NameError: name 'glob' is not defined

# Dettagli del profilo dei followers di primo e secondo livello

In [11]:
def userDetails(userList):
    
    for user in userList:
        
        raw_data = api.get_user(user)
        json_data = raw_data._json
        profile_data = {}
        profile_data["id"] = json_data["id"]
        profile_data["name"] = json_data["name"]
        profile_data["screen_name"] = json_data["screen_name"]
        profile_data["location"] = json_data["location"]
        profile_data["description"] = json_data["description"]
        profile_data["followers_count"] = json_data["followers_count"]
        profile_data["friends_count"] = json_data["friends_count"]
        serialize_json(data_folder+"/profiledetails", "1_"+profile_data["screen_name"]+"_profile_data.json", profile_data)
        pp.pprint(profile_data)
        
#userDetails(l1_followers)
#userDetails(l2a_followers_mizzaro)
#userDetails(l2a_followers_miccighel)
userDetails(l2b_followers_mizzaro)
userDetails(l2b_followers_miccighel)

Data serialized to path: data/1_MaxZancanaro_profile_data.json
{'description': '',
 'followers_count': 211,
 'friends_count': 197,
 'id': 1239642882,
 'location': '',
 'name': 'Massimo Zancanaro',
 'screen_name': 'MaxZancanaro'}
Data serialized to path: data/1_CIRES_AU_profile_data.json
{'description': '',
 'followers_count': 37,
 'friends_count': 100,
 'id': 1425660262653120517,
 'location': '',
 'name': 'ARC Training Centre for Information Resilience',
 'screen_name': 'CIRES_AU'}
Data serialized to path: data/1_mccart_richard_profile_data.json
{'description': '',
 'followers_count': 52,
 'friends_count': 83,
 'id': 1020854242150453253,
 'location': '',
 'name': 'Richard McCart',
 'screen_name': 'mccart_richard'}
Data serialized to path: data/1_Miccighel__profile_data.json
{'description': 'University Life, Computer Science, Photography, Reading, '
                'Gaming, Comics. PhD Student at the University of Udine. '
                "That's me.",
 'followers_count': 330,
 'friends

# Ultimi 5 tweet di ciascun follower di primo e secondo livello

In [13]:
def getTweets(userList):
    
    for user in userList:
        
        tweets = []
        for item in tweepy.Cursor(
                api.user_timeline, 
                screen_name=user,
                tweet_mode="extended"
            ).items(5):
            raw_tweet = item._json
            tweet = {}
            tweet["id"]= raw_tweet["id"]
            tweet["created_at"] = raw_tweet["created_at"]
            tweet["full_text"] = raw_tweet["full_text"],
            tweet["favorite_count"]= raw_tweet["favorite_count"]
            tweets.append(tweet)
        serialize_json(data_folder+"/tweets", user+"_recent_tweets.json", tweets)
        pp.pprint(tweets)
        
#getTweets(l1_followers)
#getTweets(l2a_followers_mizzaro)
#getTweets(l2a_followers_miccighel)
getTweets(l2b_followers_mizzaro)
getTweets(l2b_followers_miccighel)

Data serialized to path: data/MaxZancanaro_recent_tweets.json
[{'created_at': 'Wed Nov 03 17:29:03 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @diunito: #design #ethics #trust #Worldusabilityday '
                '#WUD2021 #wudsigchi2021 #ux #wud #uxdesign #userexperience '
                '#ai #usability #uxresearc…',),
  'id': 1455950161264394251},
 {'created_at': 'Tue Sep 14 18:47:54 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @i3_FBK: Another successful initiative @Families_Share 😄\n'
                'Co-production of childcare can really improve '
                '#WorkLifeBalance and wellbeing.',),
  'id': 1437850611899379713},
 {'created_at': 'Tue Mar 09 18:15:34 +0000 2021',
  'favorite_count': 0,
  'full_text': ('RT @elme85: 📣@TonolliLinda, @emmenomade, Roberto Cibin, Amon '
                'Rapp, @MaxZancana, @vincenzodandrea and I are organizing a '
                'workshop @chitaly20…',),
  'id': 1369351166430228490},
 {'created_at': 'Sun Mar 07 09:

# Rete sociale

In [9]:
# Creazione dataframe dei Nodi
def create_node(follower_of_user):
    #lettura json dell'utente desiderato
    with open(data_folder+"/total/"+follower_of_user+".json", "r", encoding="UTF-8") as read_file:
        data = json.load(read_file)
    #aggiunta dei nodi
    followers_nodes=[]
    for result in data:
        followers_nodes.append(result)
    #df_nodi_grafo
    df_nodes = pd.DataFrame(followers_nodes)
    return df_nodes

#print(create_node('total_users'))
df_nodes=create_node('total_users')

In [ ]:
graph = nx.DiGraph(team="Elia Gazzola, Martina Moro, Cristiano Roman, Pierre Agnoletti")

for index,row in df_nodes.iterrows():
    graph.add_node(row["id"],
    id = row["id"],
    title = row["name"],
    color ="#ffff00",
    physics = False,
    name = row['name'],
    screen_name = row['screen_name'],
    location=row['location'],
    followers_count = row["followers_count"],
    following_count = row["friends_count"]
    )

def create_edges():
    with open(data_folder+"/total/total_users.json","r",encoding="UTF-8") as totalFile:
        data=json.load(totalFile)
        for user1 in data:
            for user2 in data:
                if user1!=user2:
                    friendship=api.show_friendship(source_screen_name=user1["screen_name"],target_screen_name=user2["screen_name"])
                    if friendship[1].following:
                        print(user1["screen_name"]+" --> "+user2["screen_name"])
                        graph.add_edge(user2["id"],user1["id"],follow_rec=True if friendship[0].following else False)

create_edges()

Rate limit reached. Sleeping for: 531
Rate limit reached. Sleeping for: 32
Rate limit reached. Sleeping for: 724


KeyboardInterrupt: 

In [14]:
def disegna_grafo(grafo):
    nt = Network(
        height ="100%",
        width = "100%",
        bgcolor="#222222",
        font_color="white",
        heading= grafo,
        directed=True
    )
    nt.from_nx(grafo)
    nt.barnes_hut()
    # nt.set_edge_smooth("continuous") #cambia formato di visualizzazione degli archi
    neighbor_map = nt.get_adj_list()
    for node in nt.nodes:
        node["value"] = len(neighbor_map[node["id"]])
    # print(str(node["id"])+":"+ str(node["value"]))
    nt.show("grafo.html")

disegna_grafo(graph)